In [1]:
import os
import sys
import glob
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path
from random import shuffle

/anaconda/envs/labasi/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [30]:
df1 = pd.read_csv('/Volumes/IMVDrive/cfdb-django/glyphs-aligned-w-std_sign-images.csv', usecols=['sign', 'glyph'])
df_group = df1.groupby(by=['sign'])
df_group = sorted(df_group, key=lambda x: len(x[1])) #https://stackoverflow.com/questions/22291395/sorting-the-grouped-data-as-per-group-size-in-pandas

train_list = pd.DataFrame(data=None, columns=['sign', 'glyph','onehot'])
val_list = pd.DataFrame(data=None, columns=['sign', 'glyph','onehot'])
test_list = pd.DataFrame(data=None, columns=['sign', 'glyph','onehot'])

onehot_ids_list = []

group_count = 0
print("Thresholding number of glyphs per sign at 50...")
first = True
for name,group in df_group:
    if len(group) >= 50:
        group_count = group_count+1
        onehot_ids_list.append([group_count, name])
        
        col = []
        for g in range(len(group)):
            col.append(group_count)
        col_df = pd.DataFrame(col, columns=['onehot'])
        
        train_col = col[0:int(0.9*len(group))]
        val_col = col[int(0.9*len(group)):int(0.95*len(group))]
        test_col = col[int(0.95*len(group)):]
        
        train_group = group[0:int(0.9*len(group))] 
        val_group = group[int(0.9*len(group)):int(0.95*len(group))]
        test_group = group[int(0.95*len(group)):]
        
        train_group = train_group.assign(onehot=train_col)
        val_group = val_group.assign(onehot=val_col)
        test_group = test_group.assign(onehot=test_col)
        
        train_list = pd.concat([train_list, train_group], join_axes=[train_list.columns], ignore_index=True)
        val_list = pd.concat([val_list, val_group], join_axes=[val_list.columns], ignore_index=True)
        test_list = pd.concat([test_list, test_group], join_axes=[test_list.columns], ignore_index=True)
print("Thresholding finished.")




Thresholding number of glyphs per sign at 50...
Thresholding finished.
    sign                                     glyph onehot
0     KA  b18b7d99-82d8-46c1-880e-635a9042853a.png      1
1     KA  91b43952-c5fd-4b96-8384-2379dbc6da32.png      1
2     KU  c32e9d17-795e-45e6-b8ea-0ab2bdd04d63.png      2
3     KU  1c0c61d4-cd7e-485e-bfde-058db0d0b521.png      2
4     KU  73061f43-d4bc-4740-90e7-00fce0de8099.png      2
5    BAR  79115591-dc6d-4040-9092-590e7b59bfdc.png      3
6    BAR  37447f10-a331-4e5d-a874-7dec4690c474.png      3
7    BAR  d655addf-4aea-4ee6-9cf0-2aef9f33f766.png      3
8     ŠE  6a2cbd0b-ee3d-418a-b13b-d271d8bb6a40.png      4
9     ŠE  e1766fd1-95c5-4075-89af-037908826474.png      4
10    ŠE  033521d2-acdc-4295-a86b-97663d9eb735.png      4
11    SU  56671b4f-f7bb-4939-984d-8d8b022e2fed.png      5
12    SU  0ca38236-edb7-46bf-af22-c01a21f86d9a.png      5
13    SU  e62be7bb-fdff-4bca-a222-c3d12ff6b125.png      5
14    GI  75d3f259-8667-42d7-9830-6a7c5d95968b.png      6
1

In [3]:
## ------------------------------------------------------------
## FORMAT CUTOM IMAGE DATA SET
## --source: 
## Daniel Persson,'How to load a custom dataset with tf.data [Tensorflow]',
## https://www.youtube.com/watch?v=bqeUmLCgsVw
## ------------------------------------------------------------
## I use PIL instead of cv2
## ------------------------------------------------------------

print("FORMAT CUSTOM IMAGE INPUT")

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def createDataRecord(outFileName, addrs, labels):
    print("")
    print('Creating '+outFileName+'...')
    writer = tf.python_io.TFRecordWriter(outFileName)
    for i in range(len(addrs)):
        if not i % 1000:
            print(outFileName+' data: {}/{}'.format(i,len(addrs)))
            sys.stdout.flush() 
        filename = os.fsdecode('/Volumes/imvDrive/cfdb-django/media/glyph_img/'+addrs.iloc[i][0])
        if Path(filename).is_file():
            try:
                img = Image.open(filename).convert('L') #convert to grayscale
                img = img.resize((331,331)) 
                label = labels.iloc[i][0]
                ascii_label = int(''.join(str(ord(c)) for c in name))
                
                feature_img = {
                    'glyph_img_raw': _bytes_feature(img.tobytes()),
                    'label_raw': _int64_feature(onehot_ids_list[ascii_label])
                }

                example = tf.train.Example(features=tf.train.Features(feature=feature_img))
                writer.write(example.SerializeToString())
                
            except Exception:
                print("Corrupted record...")
                pass
    writer.close()
    sys.stdout.flush()
    
train_addrs = pd.read_csv('/Volumes/IMVDRIVE/cfdb-django/media/train_batch.csv', usecols=['glyph'])
train_labels = pd.read_csv('/Volumes/IMVDRIVE/cfdb-django/media/train_batch.csv', usecols=['sign'])
val_addrs = pd.read_csv('/Volumes/IMVDRIVE/cfdb-django/media/validation_batch.csv', usecols=['glyph'])
val_labels = pd.read_csv('/Volumes/IMVDRIVE/cfdb-django/media/validation_batch.csv', usecols=['sign'])
test_addrs = pd.read_csv('/Volumes/IMVDRIVE/cfdb-django/media/testing_batch.csv', usecols=['glyph'])
test_labels = pd.read_csv('/Volumes/IMVDRIVE/cfdb-django/media/testing_batch.csv', usecols=['sign'])

train_num_examples = len(train_addrs)
val_num_examples = len(val_addrs)
test_num_examples = len(test_addrs)

createDataRecord('train.tfrecords', train_addrs, train_labels)
createDataRecord('val.tfrecords', val_addrs, val_labels)
createDataRecord('test.tfrecords', test_addrs, test_labels)

print("DONE")

FORMAT CUSTOM IMAGE INPUT

Creating train.tfrecords...
train.tfrecords data: 0/2844
Corrupted record...
Corrupted record...
Corrupted record...
Corrupted record...
train.tfrecords data: 1000/2844
Corrupted record...
Corrupted record...
Corrupted record...
train.tfrecords data: 2000/2844
Corrupted record...
Corrupted record...

Creating val.tfrecords...
val.tfrecords data: 0/158
Corrupted record...

Creating test.tfrecords...
test.tfrecords data: 0/179
DONE


In [ ]:
sess = tf.Session()

l = [1, 2, 3, 4]
o = tf.one_hot(l, depth=len(l)+1)
print(sess.run(o))
print(o.shape)